# 创建和评估解决方案<a class="anchor" id="top"></a>

在本笔记本中，您将使用 Amazon Personalize 训练多种模型，特别是：

1. 用户个性化 – 哪些项目与特定用户的相关性最高。
1. 类似项目 – 指定一个项目，哪些项目与之类似。
1. 个性化排序 – 指定用户和项目集合，它们以什么顺序最相关。

## 综述

1. [简介](#intro)
1. [创建解决方案](#solutions)
1. [评估解决方案](#eval)
1. [使用评估指标](#use)
1. [存储有用的变量](#vars)

## 简介<a class="anchor" id="intro"></a>

总的来说，Amazon Personalize 中的算法（称为配方）在很大程度上寻求解决不同的任务，如下所述：

1. **用户个性化** – 支持 ALL HRNN 工作流程/满足用户个性化需求的新版本，我们将在此处使用该版本。
1. **HRNN 和 HRNN-Metadata** – 根据以前用户与项的交互推荐项目。
1. **HRNN-Coldstart** – 推荐尚未提供交互数据的新项目。
1. **Personalized-Ranking** – 获取一个项目集合，然后使用类似 HRNN 的方法按照可能感兴趣的顺序对它们进行排序。
1. **SIMS（类似项目）**– 根据给定的项目，推荐用户也可交互的其他项目。
1. **Popularity-Count** – 如果 HRNN 或 HRNN-Metadata 没有答案，则推荐最受欢迎的项目 – 这是默认返回值。

无论使用案例如何，所有算法都共享基于用户-项目-交互数据的学习基础，该数据由 3 个核心属性定义：

1. **UserID** – 参与交互的用户
1. **ItemID** – 用户与之交互的项目
1. **Timestamp** – 交互发生的时间

我们还支持按以下值定义的事件类型和事件值：

1. **事件类型** – 事件的分类标签（浏览、已购买、已评价等）。
1. **事件值** – 与发生的事件类型对应的值。一般来说，我们在事件类型中寻找 0 和 1 之间的归一化值。例如，如果完成交易有三个阶段（点击、添加到购物车和购买），则每个阶段的事件值分别为 0.33、0.66 和 1.0。

事件类型和事件值字段是附加数据，可用于筛选为训练个性化模型而发送的数据。在这个特定的练习中，我们没有事件类型或事件值。

要运行本笔记本，您需要运行之前的笔记本 `01_Validating_and_Importing_User_Item_Interaction_Data` 和 `02_Validating_and_Importing_Item_Metadata.ipynb`，您在笔记本中创建了一个数据集，并将交互和项目元数据导入到了 Amazon Personalize 中。在本笔记本的末尾，您保存了一些变量值，现在您需要将这些值加载到此笔记本中。

In [ ]:
%store -r

## 创建解决方案<a class="anchor" id="solutions"></a>
[返回页首](#top)

在本笔记本中，您将使用以下配方创建解决方案：

1. 用户个性化
1. SIMS
1. Personalized-Ranking

Popularity-Count 配方是 Amazon Personalize 中最简单的解决方案，只能用作备用解决方案，因此本笔记本中将不予介绍。

与之前的笔记本类似，首先导入相关软件包，然后使用 SDK 设置一个与 Amazon Personalize 的连接。

In [ ]:
import time
from time import sleep
import json

import boto3

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

在 Amazon Personalize 中，算法的特定变体称为配方。不同的配方适用于不同的情况。经过训练的模型称为解决方案，每个解决方案可以有许多版本，这些版本与模型训练时给定的数据量相关。

首先，我们将列出支持的所有配方。这将允许您选择一个配方并使用它来构建模型。

In [ ]:
personalize.list_recipes()

输出只是简介中提到的所有算法的 JSON 表示。

接下来，我们将选择特定的配方并使用它们构建模型。

### 用户个性化
User-Personalization（aws-user-personalization）配方针对所有 USER_PERSONALIZATION 推荐方案进行了优化。在推荐项目时，它使用自动项目探索。

借助自动探索功能，Amazon Personalize 自动测试不同的项目推荐，了解用户如何与这些推荐项目交互，并改进可提高参与度和转化率的项目的推荐。当您的目录快速变化，或者当新项目（如新闻文章或促销）与新用户更相关时，这可改进项目发现和参与度。

您可以平衡探索的量（推荐更频繁地探索较低交互数据或相关性较低的项目）与要利用的量（推荐基于我们的了解内容或相关性）。Amazon Personalize 根据隐式用户反馈自动调整未来的推荐。

首先，通过在上面的配方列表中找到 ARN 来选择配方。

In [ ]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### 创建解决方案

首先，使用配方创建一个解决方案。尽管您在此步骤中提供了数据集 ARN，但模型尚未经过训练。将模型视为标识符，而不是经过训练的模型。

In [ ]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "personalize-poc-userpersonalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']


In [ ]:
print(json.dumps(user_personalization_solution_arn, indent=2))

#### 创建解决方案版本

拥有解决方案之后，您需要创建一个版本来完成模型训练。训练过程可能需要一段时间才能完成（超过 25 分钟），并且使用我们的数据集进行训练时，采用此配方时平均需要 90 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，这里的目标是创建许多模型并快速部署它们。因此，我们将为笔记本中的所有解决方案设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [ ]:
userpersonalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

In [ ]:
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

### SIMS


SIMS 是 Amazon 中用于推荐系统的最旧算法之一。它的核心使用案例为如下情况：当您有一个项目，并且您想要推荐在整个用户群中以类似方式与之交互的项目时。这意味着结果并没有针对每个用户进行个性化设置。有时，这会导致推荐最受欢迎的项目。因此，有一个可以进行调整的超参数，这将减少结果中受欢迎项目的数量。

对于我们的使用案例，使用 Movielens 数据，假设我们选择了某部电影。然后，我们可以使用 SIMS，根据整个用户群的交互行为推荐其他电影。结果没有针对每个用户进行个性化设置，而是根据我们选作输入内容的电影而有所不同。

就像上次一样，我们从选择配方开始。

In [ ]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### 创建解决方案

与 HRNN 一样，首先创建解决方案。尽管您在此步骤中提供了数据集 ARN，但模型尚未经过训练。将模型视为标识符，而不是经过训练的模型。

In [ ]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

#### 创建解决方案版本

拥有解决方案之后，您需要创建一个版本来完成模型训练。使用我们的数据集，完成训练可能需要一段时间，最多 25 分钟，平均 35 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，这里的目标是创建许多模型并快速部署它们。因此，我们将为笔记本中的所有解决方案设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [ ]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [ ]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

### 个性化排名

个性化排名是 HRNN 的一个有趣应用。该算法不仅为相关用户推荐最有可能的内容，还接受用户和项目列表。然后按照可能与用户最相关的顺序重新呈现这些项目。此处的使用案例用于筛选您没有项目元数据来创建筛选器的唯一类别，或者当您有一个广义的集合，您希望为特定用户更好地进行排序时。

在我们的使用案例中，使用 MovieLens 数据，我们可以想象 VOD 应用程序可能想要创建一系列由漫画改编的电影或由特定导演制作的电影。我们很可能拥有这些基于列表的标题元数据。我们将使用个性化排序，根据每个用户之前的标记历史记录，为用户重新排序电影列表。

就像上次一样，我们从选择配方开始。

In [ ]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### 创建解决方案

与之前的解决方案一样，首先创建解决方案。尽管您在此步骤中提供了数据集 ARN，但模型尚未经过训练。将模型视为标识符，而不是经过训练的模型。

In [ ]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

#### 创建解决方案版本

拥有解决方案之后，您需要创建一个版本来完成模型训练。使用我们的数据集，完成训练可能需要一段时间，最多 25 分钟，平均 35 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，这里的目标是创建许多模型并快速部署它们。因此，我们将为笔记本中的所有解决方案设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [ ]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [ ]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

### 查看解决方案创建状态

如前面所述，如何在控制台中查看状态更新呢：

* 在另一个浏览器选项卡中，您应该已经在打开此笔记本实例时启动了 AWS 控制台。
* 切换到该选项卡并在顶部搜索服务 `Personalize`，然后转到该服务页面。
* 单击 `View dataset groups`。
* 单击数据集组的名称，名称中很可能包含 POC。
* 单击 `Solutions and recipes`。
* 现在，您将看到您在上面创建的所有解决方案的列表，包括一个包含解决方案版本状态的列。在状态变成 `Active` 之后，您的解决方案就可以接受审核了。它也能够被部署。

也可简单地运行下面的单元格以跟踪解决方案版本的创建状态。

In [ ]:
in_progress_solution_versions = [
    userpersonalization_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

### 超参数调整

个性化提供了创建解决方案时运行超参数调整的选项。由于执行超参数调整需要另行计算，此功能默认情况下处于关闭状态。因此，我们在上面创建的解决方案将只使用每个配方的默认超参数值。有关超参数调整的更多信息，请参阅[文档](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html)。

如果您已经确定了要使用的正确配方，并且准备运行超参数调整，则以下代码将显示如何进行此调整，使用 SIMS 作为示例。

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    performHPO=True
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

如果您已知要用于特定超参数的值，也可以在创建解决方案时设置此值。以下代码显示如何为 SIMS 配方设置 `popularity_discount_factor` 值。

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'popularity_discount_factor': '0.7'
        }
    }
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

## 评估解决方案版本<a class="anchor" id="eval"></a>
[返回页首](#top)

通过该笔记本训练所有的解决方案应该不会超过一个小时。在训练过程中，我们建议花时间详细阅读各种算法（配方）及其行为。这也是一个绝佳时机，可以考虑如何将数据输入系统以及您希望看到的结果的替代方案。

解决方案创建完成后，下一步是获取评估指标。Personalize 根据训练数据的子集计算这些指标。下图说明了 Personalize 拆分数据的方式。给定 10 个用户，每个用户有 10 次交互（一个圆圈代表一次交互），交互根据时间戳从最早到最晚进行排序。Personalize 使用来自 90% 用户的所有交互数据（蓝色圆圈）来训练解决方案版本，剩下的 10% 用于评估。对于剩余 10% 中的每个用户，用户 90% 的交互数据（绿色圆圈）被用作调用训练模型的输入。其余 10% 的数据（橙色圆圈）与模型产生的输出进行比较，并用于计算评估指标。

![个性化指标](static/imgs/personalize_metrics.png)

我们建议阅读[文档](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)了解指标，但为了方便起见，我们还复制了以下文档的部分内容。

您需要了解 Personalize 中有关评估的以下术语：

* *相关推荐*是指与特定用户的测试数据中的值相匹配的推荐。
* *排名*是指推荐项目在推荐列表中的位置。位置 1（列表顶部）假定与用户最相关。
* *查询*是指 GetRecommendations 调用的内部等效项。

Personalize 生成的指标包括：
* **coverage**: 训练数据（包括项目和交互数据集）中来自所有查询的唯一推荐项目数占唯一项目总数的比例。
* **mean_reciprocal_rank_at_25**: 对所有查询提出的前 25 项推荐中，第一项相关推荐的[倒数排名的平均值](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)。如果您对单个排名最高的推荐感兴趣，则此指标是合适的。
* **normalized_discounted_cumulative_gain_at_K**: 折损增益假设推荐列表中较下面的推荐的相关性低于较上面的推荐。因此，每项推荐都被一个因数折损（为推荐赋予较低的权重），具体取决于推荐的位置。为了产生 K 处的[累积折损增益](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)（DCG），将前 K 个推荐中的每个相关折损推荐加在一起。归一化折损累积增益（NDCG）是 DCG 除以理想 DCG，使得 NDCG 在 0-1 之间。（理想 DCG 是按相关性对前 K 个推荐进行排序。） Amazon Personalize 使用 1/log(1 + position) 权重因子，其中列表顶部为位置 1。此指标奖励出现在列表顶部附近的相关项目，因为列表的顶部通常会更受关注。
* **precision_at_K**: 前 K 个推荐中相关推荐的数量除以 K。此指标奖励相关项目的精确推荐。

让我们来看看此笔记本中生成的每个解决方案的评估指标。* 请注意，由于 Movielens 数据集的质量不同，您的结果可能与本笔记本文本中描述的结果不同。* 

### 用户个性化指标

首先，检索用户个性化解决方案版本的评估指标。

In [ ]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)

print(json.dumps(user_personalization_solution_metrics_response, indent=2))

以上标准化的累计折损增益向我们表明，在 5 个项目中，作为用户交互历史记录的一部分，我们在推荐（训练和验证的暂停阶段）中的机会小于 22%（对于完整数据集）/38%（对于较小版本数据集）。大约 13% 的推荐项目是独一无二的，我们在前 5 个推荐项目中的精确度仅为 14%（对于完整数据集）/7.5%（对于较小版本数据集）。

这显然并不是一个很好的模型，但请记住，我们必须使用评级数据进行交互，因为 Movielens 是基于评级的显式数据集。时间戳也是从电影评级时而非观看时开始的，因此该顺序与观看者观看电影的顺序不同。

### SIMS 指标

现在，检索 SIMS 解决方案版本的评估指标。

In [ ]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

在本例中，我们看到 5 个项目的精确度略有提高，此时略高于 4.5%（对于完整数据集）/6.4%（对于较小版本数据集）。实际上，这可能在误差范围内，但鉴于没有努力掩盖人气，可能只是返回大量用户以某种方式进行交互的超级受欢迎的结果。

### 个性化排序指标

现在，检索个性化排序解决方案版本的评估指标。

In [ ]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

只是简要概述一下，在此我们再次看到精确度接近 2.7%（对于完整数据集）/2.2%（对于较小版本数据集），因为精确度基于用户个性化，这在意料之中。然而，验证的样本项目不同，因此评级较低。

## 使用评估指标<a class="anchor" id="use"></a>
[返回页首](#top)

请务必谨慎使用评估指标。要牢记多个因素。

* 如果现有的推荐系统已到位，将影响您用于训练新解决方案的用户交互历史记录。这意味着评估指标偏向于现有解决方案。如果您努力使评估指标匹配或超越现有解决方案，那么您可能只是推动 User Personalization，使其表现与现有解决方案相似，并且最终可能无法获得更好的效果。
* HRNN Coldstart 配方难以使用 Amazon Personalize 生成的指标进行评估。此配方的目的是推荐您的业务中的新项目。因此，这些项目不会显示在用于计算评估指标的现有用户事务数据中。因此，仅在对比评估指标时，HRNN Coldstart 的表现绝不会比其他配方更好。注意：User Personalization 配方还包括改进的冷启动功能

考虑到这些因素，Personalize 生成的评估指标通常适用于以下两种情况：
1. 比较接受过相同配方训练、但具有不同的超参数和功能值的解决方案版本的性能（展示数据等）
1. 比较接受过不同配方训练的解决方案版本的性能（HRNN Coldstart 除外）。

正确评估推荐系统最好始终通过 A/B 测试来完成，同时衡量实际的业务成果。由于系统生成的推荐通常会影响推荐所基于的用户行为，因此最好运行小实验，并在较长的时间内应用 A/B 测试。随着时间的推移，来自现有模型的偏差将逐渐消失。

## 存储有用的变量<a class="anchor" id="vars"></a>
[返回页首](#top)

在退出本笔记本之前，请运行以下单元格保存版本 ARN，以便在下一个笔记本中使用。

In [ ]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn